In [1]:
import numpy as np
from scipy.stats import invgamma

In [48]:
import numpy as np
import scipy.stats as stats

data = np.array([529, 530, 532, 533.1, 533.4, 533.6, 533.7, 534.1, 534.8, 535.3, 
535.4, 535.9, 536.1, 536.3, 536.4, 536.6, 537, 537.4, 537.5, 
538.3, 538.5, 538.6, 539.4, 539.6, 540.4, 540.8, 542, 542.8, 
543, 543.5, 543.8, 543.9, 545.3, 546.2, 548.8, 548.7, 548.9, 
549, 549.4, 549.9, 550.6, 551.2, 551.4, 551.5, 551.6, 552.8, 
552.9, 553.2])
N=48

# Initialize parameters
K = 2  # Number of clusters
mu = np.array([530, 540])  # Initial guesses for means
sigma = np.array([1, 1])  # Initial guesses for standard deviations
pi = np.array([0.5, 0.5])  # Initial guesses for mixing coefficients



# Define functions for the Gibbs sampler
def sample_z(data, mu, sigma, pi):
    """Sample cluster assignments."""
    z = np.zeros(len(data), dtype=int)
    for i, x in enumerate(data):
        p = pi * stats.norm.pdf(x, loc=mu, scale=sigma)
        p /= np.sum(p)
        z[i] = np.random.choice(np.arange(len(pi)), p=p)
    return z

def sample_mu(data, z, K, mu, tau):
    """Sample means."""
    lambda_1=mu[0]
    theta = mu[1]-mu[0]
    sigma_l=1
    mu_l=1
    sigma_t=1
    mu_t=1
    mu_samples = np.zeros(K)
    partie1 = 1/((1/sigma_l**2)+(N/tau**2))
    partie2 = (mu_l/sigma_l**2) + np.sum(data-z*theta)/tau**2
    
    
    partie3 = 1/((1/sigma_t**2)+np.sum(z)/tau**2)
    partie4 = (mu_t/sigma_t**2) + np.sum((lambda_1-data)*z)/tau**2 
    lambda_1 = np.random.normal(partie1*partie2, partie1)
    theta = abs(np.random.normal(partie3*partie4, partie3))
       
    
    mu_samples[0]=lambda_1
    mu_samples[1]=lambda_1+theta
    
    return mu_samples

def sample_sigma(data, z, mu, K):
    """Sample standard deviations."""
    alpha=1
    beta=1
    lambda_1=mu[0]
    theta=mu[1]-mu[0]
    partie1 = alpha +N/2
    partie2 = beta
    for i in range(N):
        partie2+=(data[i]-(lambda_1+z[i]*theta)**2)/2
    
    tau2 = invgamma.rvs(partie1, 1/partie2)
        
    sigma_samples = np.zeros(K)
    sigma_samples[0]=1/np.sqrt(np.sqrt(tau2))
    sigma_samples[1]=1/np.sqrt(np.sqrt(tau2))
    
    return sigma_samples

def sample_pi(z, K):
    """Sample mixing coefficients."""
    p1 = np.random.beta(1+np.sum(z),1-np.sum(z-1))    
    pi_samples = np.zeros(K)
    pi_samples[0]=p1
    pi_samples[1]=1-p1
    
    return pi_samples

# Run Gibbs sampler
n_samples = 1000
burn_in = 100
mu_samples = np.zeros((n_samples+burn_in+1, K))
mu_samples[0]=[530,550]
sigma_samples = np.zeros((n_samples+burn_in+1, K))
sigma_samples[0]=[1,1]
pi_samples = np.zeros((n_samples+burn_in+1, K))
pi_samples[0]=[0.5,0.5]

for i in range(n_samples + burn_in):
    z = sample_z(data, mu, sigma, pi)
    #if i >= burn_in+1:
    mu_samples[i+1] = sample_mu(data, z, K, mu_samples[i], sigma_samples[i][0])
    sigma_samples[i+1] = sample_sigma(data, z, mu, K)
    pi_samples[i+1] = sample_pi(z, K)

    
mu_samples=mu_samples[burn_in+1:]
sigma_samples=sigma_samples[burn_in+1:]
pi_samples=pi_samples[burn_in+1:]
# Print estimated parameters
print("Estimated Means:", mu_samples.mean(axis=0))
print("Estimated Standard Deviations:", sigma_samples.mean(axis=0))
print("Estimated Mixing Coefficients:", pi_samples.mean(axis=0))


Estimated Means: [391.70967651 526.39753785]
Estimated Standard Deviations: [2.22245647 2.22245647]
Estimated Mixing Coefficients: [0.79939315 0.20060685]
